In [ ]:
!pip install datasets
from datasets import load_dataset
dataset = load_dataset(
    'amazon_us_reviews', 'Mobile_Electronics_v1_00')

In [2]:
import pandas as pd
data = dataset['train'][:]
df = pd.DataFrame(data, columns=data.keys())
training_data = df.sample(frac=0.8, random_state=25)
testing_data = df.drop(training_data.index)
print(f"No. of training examples: {training_data.shape[0]}")
print(f"No. of testing examples: {testing_data.shape[0]}")

No. of training examples: 83980
No. of testing examples: 20995


In [18]:
import csv
reviewsAndRating = open("./reviewsAndRating.csv", 'w')
allReviews = csv.writer(reviewsAndRating)
allReviews.writerow(["review", "rating"])

15

In [19]:
for row in training_data.iterrows():
    print(row)
    stars = row[1].star_rating
    review = row[1].review_body
    allReviews.writerow([review, stars])

In [32]:
from nltk.tokenize import NLTKWordTokenizer
import pandas

reviewsAndRating = open("./reviewsAndRating.csv", 'r')
annotated_reviews_csv  = pandas.read_csv(reviewsAndRating, delimiter = ",", usecols=["review", "rating"])
tknzr = NLTKWordTokenizer()
bowListTrain = []
for row in annotated_reviews_csv.iterrows():
    bow = {}
    tokens = tknzr.tokenize(str(row[1].review).lower())
    for token in tokens:
        if bow.get(token)!=None:
            bow[token] +=1
        else:
            bow[token] = 1
    bowListTrain.append(bow)

In [36]:
print(len(bowListTrain))

83987


In [40]:
reviewsAndRatingTest = open("./reviewsAndRatingsTest.csv", 'w')
allReviewsTest = csv.writer(reviewsAndRatingTest)
allReviewsTest.writerow(["review", "rating"])

15

In [41]:
for row in testing_data.iterrows():
    stars = row[1].star_rating
    review = row[1].review_body
    allReviewsTest.writerow([review, stars])

In [43]:
from nltk.tokenize import NLTKWordTokenizer
import pandas

reviewsAndRatingTest = open("./reviewsAndRatingsTest.csv", 'r')
annotated_reviews_csv_test = pandas.read_csv(reviewsAndRatingTest, delimiter=",", usecols=["review", "rating"])
tknzr = NLTKWordTokenizer()
bowListTest = []
for row in annotated_reviews_csv_test.iterrows():
    bow = {}
    tokens = tknzr.tokenize(str(row[1].review).lower())
    for token in tokens:
        if bow.get(token) != None:
            bow[token] += 1
        else:
            bow[token] = 1
    bowListTest.append(bow)

In [44]:
print(len(bowListTest))


20988


In [50]:
from nltk.tokenize import NLTKWordTokenizer
import pandas

reviewsAndRatingTest = open("./reviewsAndRatingsTest.csv", 'r')
annotated_reviews_csv_test_reviews = pandas.read_csv(reviewsAndRatingTest, delimiter=",", usecols=["review"])
bowListTestReview = []
for row in annotated_reviews_csv_test_reviews.iterrows():
    bowListTestReview.append(str(row))

In [71]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.feature_extraction import DictVectorizer
from sklearn import *
v = DictVectorizer()
X_train = v.fit_transform(bowListTrain)
X_predict = v.transform(bowListTest)

print(X_train.shape)
print(X_predict.shape)


(83987, 83248)
(20988, 83248)


/Users/elinottonson/.conda/envs/490A/lib/python3.9/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass Cs=l2 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
/Users/elinottonson/.conda/envs/490A/lib/python3.9/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"


TypeError: unsupported operand type(s) for /: 'float' and 'str'

In [72]:

y = annotated_reviews_csv["rating"]
model = LogisticRegressionCV(max_iter=500, solver='newton-cg', cv=10, class_weight='balanced')
fitted_model = model.fit(X_train, y)
prediction = fitted_model.predict(X_predict)

/Users/elinottonson/.conda/envs/490A/lib/python3.9/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"


KeyboardInterrupt: 

In [69]:
from sklearn.metrics import classification_report
from sklearn. import classification_report
intPreds = [int(i) for i in prediction]

print(classification_report(annotated_reviews_csv_test['rating'], intPreds))


              precision    recall  f1-score   support

           1       0.68      0.74      0.71      3536
           2       0.25      0.16      0.20      1478
           3       0.34      0.26      0.30      1988
           4       0.41      0.28      0.34      3672
           5       0.75      0.88      0.81     10314

    accuracy                           0.64     20988
   macro avg       0.49      0.47      0.47     20988
weighted avg       0.60      0.64      0.62     20988



In [70]:
print(annotated_reviews_csv['rating'].value_counts())


5                                                                                          41941
4                                                                                          14415
1                                                                                          14050
3                                                                                           7746
2                                                                                           5833
 a little elbow-grease and a towel do a much better job!"                                      1
 so started looking for docking with speakers. Found this one at such a low price ($40)        1
Name: rating, dtype: int64
